In [2]:
#pip install datorama
import json
import requests
import numpy
import pandas

In [3]:
def create_api_format_pandas_df(API_KEY, data):

    """
    Function to create a Query API connection with Datorama, format result of response into JSON format
    
    params:
    API_KEY: Datorama User API Token, only enabled for API-enabled users, string format
        To find your Token, navigate to platform.datorama.com > CTRL (CMND) + K > 'My Profile' > Copy from there
    API_ENDPOINT: 'https://app.datorama.com/v1/query'
        This is a fixed parameter that cannot be changed unless guidance from Datorama Support changes
        String format
    data: Query copied by the Datorama Query API Builder found here 
        'https://platform.datorama.com/' + '[WORKSPACE ID]' + '/analyze/query-builder'
        Dictionary format
    
    returns
    json_dumped_data: json-formatted data, pulled from the API response
        String format
    """

    # Test the API_KEY has been formatted correctly 
    if not API_KEY.startswith('dato-api-'):
        raise NameError("Did you forget something?")
        
    # Pass your authenticated API_Key into headers
    headers = {'content-type': 'application/json', 'Authorization': API_KEY}
    
    # Ensure the data is in dictionary format
    if not type(data) == dict:
        raise TypeError("You had one job! Copy-pasting is NOT that hard!")
        
    # Format your dictionary "data" into a json string
    payload = json.dumps(data)
    
    # API_ENDPOINT is a fixed value that will only change when the Datorama Support Team says it does
    API_ENDPOINT = 'https://app.datorama.com/v1/query'
    
    # Pass API_ENDPOINT, headers, and payload into a requests.post function
    # We use the "post" request because this is how Datorama allows us to retrieve data
    r = requests.post(url = API_ENDPOINT, headers = headers, data = payload)
    
    # Reformat the result of your request into a json string
    json_dumped_data = r.json()
    
    # Parse the relevant response data
    response_data = json_dumped_data['queryResponseData']
    
    # Parse the keys
    keys = response_data['headers']
    
    # Create an empty list, where we will store a list of dictionaries
    final_dictionary_vals = []
    
    # Traverse each row and "restore" the data as separate dictionaries
    for i in response_data['rows']:
        final_dictionary_vals.append(dict(zip(keys,i)))
        
    # Convert to pandas
    final_pandas_df = pandas.DataFrame(final_dictionary_vals)
        
    return final_pandas_df

In [4]:
API_KEY = 'dato-api-952fadbd-ccef-4664-83e3-c5a59c488479'
data = {
	"workspaceId": "9185",
	"dateRange": "CUSTOM_ON_GOING",
	"startDate": "2019-01-01",
	"endDate": "2020-03-25",
	"measurements": [
		{
			"name": "Tiffany Capped Spend"
		},
		{
			"name": "Impressions"
		},
		{
			"name": "IO Capped Cost"
		}
	],
	"dimensions": [
		"Channel (Dashboard)",
		"Cross Channel Partner",
		"IO Name",
		"Campaign Name",
		"Media Buy Name",
		"Cross Channel Ad Format",
		"Cross Channel Target Audience",
		"Cross Channel Funnel Position"
	],
	"groupDimensionFilters": [],
	"stringDimensionFilters": [
		{
			"operator": "NOT_CONTAINS",
			"val": "search",
			"dimension": "Channel (Dashboard)"
		},
		{
			"operator": "NOT_CONTAINS",
			"val": "social",
			"dimension": "Channel (Dashboard)"
		},
		{
			"operator": "NOT_CONTAINS",
			"val": "sizmek",
			"dimension": "Data Stream"
		},
		{
			"operator": "NOT_CONTAINS",
			"val": "other",
			"dimension": "IO Key"
		},
		{
			"operator": "NOT_CONTAINS",
			"val": "catalyst",
			"dimension": "Cross Channel Partner"
		},
		{
			"operator": "NOT_CONTAINS",
			"val": "_CA_",
			"dimension": "Campaign Name"
		},
		{
			"operator": "NOT_CONTAINS",
			"val": "_Social_",
			"dimension": "Campaign Name"
		},
		{
			"operator": "CONTAINS",
			"val": "FY",
			"dimension": "Campaign Name"
		}
	],
	"stringDimensionFiltersOperator": "AND",
	"numberDimensionFiltersOperator": "AND",
	"numberMeasurementFilter": [
		{
			"val": " ",
			"operator": "IS_NOT_NULL",
			"fieldName": "Impressions"
		}
	],
	"sortBy": "Channel (Dashboard)",
	"sortOrder": "DESC",
	"topResults": "600000",
	"groupOthers": True,
	"topPerDimension": False,
	"totalDimensions": []
}

tiffany_dig_data = create_api_format_pandas_df(API_KEY, data)
print(tiffany_dig_data)

         Channel (Dashboard) Cross Channel Partner  \
0        Site Direct - Video                 Vogue   
1        Site Direct - Video     CNN International   
2        Site Direct - Video                   CNN   
3        Site Direct - Video                   CNN   
4        Site Direct - Video               YouTube   
...                      ...                   ...   
4942  Programmatic - Display                Amazon   
4943  Programmatic - Display      American Express   
4944  Programmatic - Display                Amazon   
4945  Programmatic - Display                 XAXIS   
4946  Programmatic - Display      American Express   

                                                IO Name  \
0     P13V015_TIF_GBL_017_Content_VOGUE_D_ISV_CPM_NA...   
1     P1766Y0_TIF_GBL_026_Content_CNN INTERNATIONAL_...   
2     P1768CP_TIF_GBL_026_Content_CNN_ALL_DIS_FLF_NA...   
3     P17611D_TIF_GBL_026_Content_CNN_ALL_DIS_FLF_NA...   
4     P14R37B_TIF_GBL_023_Content_YOUTUBE.COM_ALL_IS... 